<a href="https://colab.research.google.com/github/beny2000/HeartDiseaseClassifier/blob/main/Final_SVM_JJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Connect to Anvil
!pip install anvil-uplink
import anvil.server
anvil.server.connect("2ZZFEIO7RSWJ2NSXVOIJU7NK-PUMVCYIB7OL5ZPSL")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


In [2]:
#Import scikit-learn dataset library
from sklearn import datasets
import pandas as pd
import numpy as np


from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

#Load dataset
cleveland = pd.read_csv('heart.csv')

In [3]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
x = np.array(cleveland.drop(['target'], 1))
y = np.array(cleveland['target'])

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [4]:
#Import svm model
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(C = 60, kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [5]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
Accuracy = metrics.accuracy_score(y_test, y_pred)
print("Accuracy:",Accuracy)

Accuracy: 0.8852459016393442


In [6]:
#Create Callable Function
@anvil.server.callable
def svmpredict(Age, Sex, CP, BP, Cholesterol, BS, RestECG_Results, MaxHR, exind_Angina, ST_Depression, ST_Slope, MajorVessels, Thal_Test):
  #Converts to CSV file
  patientData = pd.DataFrame([[Age, Sex, CP, BP, Cholesterol, BS, RestECG_Results, MaxHR, exind_Angina, ST_Depression, ST_Slope, MajorVessels, Thal_Test]], columns=['Age','Sex','CP','BP','Cholesterol','BS', 'RestECG_Results','MaxHR', 'exind_Angina', 'ST_Depression','ST_Slope','MajorVessels','Thal_Test'])
  patientData.to_csv('patientData.csv')
  
  #Calculates results
  PatientTest = clf.predict(patientData)
  PercentAccuracy = round(Accuracy * 100, 2)
  return PatientTest, PercentAccuracy


In [ ]:
anvil.server.wait_forever()